In [1]:
import sys
sys.path.insert(0,'C:\\code\\masters_thesis\\')

"""
Magic commands
"""

%load_ext autoreload
%autoreload 2

"""
Load relevant packages
"""

import numpy as np
import pandas as pd
import requests
import random
import bs4 as bs
import datetime
import yfinance as yf
# from pandas_datareader import data as pdr
# from pandas_datareader.data import DataReader
from pandas_datareader.famafrench import FamaFrenchReader
from pandas_datareader.yahoo.daily import YahooDailyReader
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.optimize import minimize
import array_to_latex as a2l

"""
Own packages
"""

# from codelib.statistics.moments import corr_to_cov_matrix
from codelib.population_parameters import calculate_mean, calculate_cov_mat
from codelib.portfolio_measures import portfolio_mean, portfolio_variance, portfolio_std
from codelib.portfolio_performance_measures import sharpe_ratio

# Start and end date

In [2]:
start_date = datetime.datetime(1991, 1, 1)
end_date = datetime.datetime(2023, 1, 1)

# Riskfree rate

In [4]:
# Riskfree rate
# get one-month US T-bill rate
reader = FamaFrenchReader("F-F_Research_Data_Factors", start=start_date)
ff_factors = reader.read()

# print details
print(ff_factors['DESCR'])

# get monthly risk-free rate (US one month T-Bill return)
us_tbill = pd.DataFrame(ff_factors[0]['RF'])
us_tbill.to_csv(r'C:\code\masters_thesis\data\us_onemonth_tbill_returns.csv')

# Fama-French 10 Industry portfolios

In [7]:
# load data
reader = FamaFrenchReader("10_Industry_Portfolios", start=start_date, end=end_date)
ind_port = reader.read()

# print details
print(ind_port['DESCR'])

# get value and equally weighted monthly industry portfolio returns
ind_mc_weighted = ind_port[0] #value weighted
ind_eq_weighted = ind_port[1] #equally weighted

# remove whitespace form column names
ind_eq_weighted.columns = [_.strip() for _ in ind_eq_weighted.columns]
ind_mc_weighted.columns = [_.strip() for _ in ind_mc_weighted.columns]
assets = np.array(ind_mc_weighted.columns)

# get monthly return premiums
ind_mc_weighted_premiums = ind_mc_weighted.sub(us_tbill['RF'], axis=0)

ind_mc_weighted.to_csv(r'C:\code\masters_thesis\data\10industry_mc_monthly_returns.csv')
ind_mc_weighted_premiums.to_csv(r'C:\code\masters_thesis\data\10industry_mc_monthly_returnpremiums.csv')

10 Industry Portfolios
----------------------

This file was created by CMPT_IND_RETS using the 202212 CRSP database. It contains value- and equal-weighted returns for 10 industry portfolios. The portfolios are constructed at the end of June. The annual returns are from January to December. Missing data are indicated by -99.99 or -999. Copyright 2022 Kenneth R. French

  0 : Average Value Weighted Returns -- Monthly (384 rows x 10 cols)
  1 : Average Equal Weighted Returns -- Monthly (384 rows x 10 cols)
  2 : Average Value Weighted Returns -- Annual (32 rows x 10 cols)
  3 : Average Equal Weighted Returns -- Annual (32 rows x 10 cols)
  4 : Number of Firms in Portfolios (384 rows x 10 cols)
  5 : Average Firm Size (384 rows x 10 cols)
  6 : Sum of BE / Sum of ME (32 rows x 10 cols)
  7 : Value-Weighted Average of BE/ME (32 rows x 10 cols)


# 20 random stocks from S&P500

In [ ]:
# get wikipedia table containing S&P 500 tickers
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

# get S&P500 tickers and remove \n
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)
tickers = [s.replace('\n', '') for s in tickers]

# choose 20 tickers randomly
random.seed(1)
random_tickers = random.choices(tickers, k=20)
print(random_tickers)

ConnectionError: HTTPConnectionPool(host='en.wikipedia.org', port=80): Max retries exceeded with url: /wiki/List_of_S%26P_500_companies (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000129685ADA90>: Failed to establish a new connection: [WinError 10060] Et forsøg på at oprette forbindelse mislykkedes, fordi den part, der havde oprettet forbindelse, ikke svarede korrekt efter en periode, eller en oprettet forbindelse blev afbrudt, fordi værten ikke svarede'))

In [ ]:
# find random tickers with no 'nan's in timeperiod:



In [10]:
# get 'adjusted close' stock prices from 20 random stocks in S&P500 (tickers found above) CORRECT THIS!!!!
tickers = ['ACGL','ADM','AIG','ALK','AOS','BSX','CL','CPRT','EXC','GE','HD','HIG','HST','JBHT','KO','MAS','MGM','SJM','SNA','TROW']

start_date = datetime.date(1996, 1, 1)
end_date = datetime.date(2022, 1, 2)
SP500_adj_close = yf.download(tickers, start=start_date, end=end_date, interval='1mo')['Close']

# remove time from datetime index
SP500_adj_close.index = pd.to_datetime(SP500_adj_close.index.date)

# calculate monthly returns (note: index is one month ahead, maybe change startdate to 31-12-1995?)
SP500_monthly_returns = ((SP500_adj_close / SP500_adj_close.shift(1) - 1) * 100).dropna()
SP500_monthly_returns.to_csv(r'C:\code\masters_thesis\data\random_SP500_monthly_returns.csv')

[*********************100%***********************]  20 of 20 completed
